In [83]:
import requests
import json
import base64
import PIL
from PIL import Image
import pandas as pd
import time

In [55]:
for stock_image in os.listdir("stock_images"):
    if stock_image != ".DS_Store" and "transposed" not in stylegan_image:
         #read the image
        im = Image.open(f"stock_images/{stock_image}")
        #get the ratio, make sure longest edge is 2000 pixels and shorter edge is proportional
        width, height = im.size
        print(f"Old dimensions: ({width}, {height})")
        if width > height:
            ratio = height / width
            new_width = 2000
            new_height = int(ratio * new_width)
        else:
            ratio = width / height
            new_height = 2000
            new_width = int(ratio * new_height)
        newsize = (new_width, new_height)
        im = im.resize(newsize)
        print(f"New dimensions: ({new_width}, {new_height})")
        im.save(f"stock_images/{stock_image}")

Old dimensions: (3231, 3231)
New dimensions: (2000, 2000)
Old dimensions: (3196, 3196)
New dimensions: (2000, 2000)
Old dimensions: (1984, 1984)
New dimensions: (2000, 2000)
Old dimensions: (4106, 4106)
New dimensions: (2000, 2000)
Old dimensions: (2824, 2818)
New dimensions: (2000, 1995)
Old dimensions: (2643, 2643)
New dimensions: (2000, 2000)
Old dimensions: (2875, 2859)
New dimensions: (2000, 1988)
Old dimensions: (4158, 4237)
New dimensions: (1962, 2000)
Old dimensions: (3374, 3374)
New dimensions: (2000, 2000)
Old dimensions: (2533, 2533)
New dimensions: (2000, 2000)
Old dimensions: (2820, 2820)
New dimensions: (2000, 2000)
Old dimensions: (3313, 3313)
New dimensions: (2000, 2000)
Old dimensions: (6438, 5187)
New dimensions: (2000, 1611)
Old dimensions: (2635, 2635)
New dimensions: (2000, 2000)
Old dimensions: (3035, 3035)
New dimensions: (2000, 2000)
Old dimensions: (4075, 3967)
New dimensions: (2000, 1946)
Old dimensions: (1943, 1943)
New dimensions: (2000, 2000)
Old dimensions

In [84]:

BASE_URL = 'https://api-us.faceplusplus.com/facepp/v3/detect'

def detect(image_path):
    """
    This function uses HTTP POST to query the Face++ API 
    that detects and classifies faces in the given image
    
    Args:
        image_path: path to the image that we want to classify
        
    Returns:
        result: a dictionary containing the API response
    """

    #read the image
    im = Image.open(image_path)
    #flip image
    out = im.transpose(PIL.Image.FLIP_LEFT_RIGHT)
    split = image_path.split("/")
    split[-1] = "transposed-" + split[-1]
    transp_image_path = "/".join(split)
    out.save(transp_image_path)
    
    
    def get_call(image_path): 
        with open(image_path, 'rb') as image_file:
            # encode in base64
            image_base64 = base64.b64encode(image_file.read())
            
            return {'api_key': "AJUC6jZcYaEtVm62lA944PD18g8qzLn-",
                'api_secret': "NiML8ojHjroKMog2AKvzMuu5PpaJhDP2",
                'image_base64': image_base64,
                'return_attributes': 'gender,age'} 
    
    
    original_response = requests.post(url = BASE_URL, data = get_call(image_path))
    flipped_response = requests.post(url = BASE_URL, data = get_call(transp_image_path))
    return json.loads(original_response.content), json.loads(flipped_response.content)

In [85]:
def get_dems_of_row(row_name):
    if row_name[-9] == "2":
        race = "black"
    else:
        race = "white"
    if row_name[-7] == "2":
        gender = " male"
    else:
        gender = " female"
    if row_name[-5] == "0":
        age = "child"
    elif row_name[-5] == "1":
        age = "teen"
    elif row_name[-5] == "2":
        age = "adult"
    elif row_name[-5] == "3":
        age = "middle-age"
    else:
        age = "old"
    return gender + " " + race + " " + age

In [86]:
stylegan_images = []
stock_images = []
for stylegan_image in os.listdir("stylegan_images"):
    if stylegan_image != ".DS_Store":
        stylegan_images.append(stylegan_image)
for stock_image in os.listdir("stock_images"):
    if stock_image != ".DS_Store":
        stock_images.append(stock_image)

In [87]:
image_dem = []
for ad in pd.Series(stylegan_images):
    new_name = get_dems_of_row(ad)
    image_dem.append("stylegan" + new_name)
for ad in pd.Series(stock_images):
    new_name = get_dems_of_row(ad)
    image_dem.append("stock" + new_name)

In [88]:
sg_original_ages = []
sg_original_genders = []
sg_y_flipped_ages = []
sg_y_flipped_genders = []
for stylegan_image in os.listdir("stylegan_images"):
    if stylegan_image != ".DS_Store" and "transposed" not in stylegan_image:
        original, flipped  = detect(f"stylegan_images/{stylegan_image}")
        sg_original_ages.append(original['faces'][0]['attributes']['age']['value'])
        sg_original_genders.append(original['faces'][0]['attributes']['gender']['value'])
        sg_y_flipped_ages.append(flipped['faces'][0]['attributes']['age']['value'])
        sg_y_flipped_genders.append(flipped['faces'][0]['attributes']['gender']['value'])
        print(len(sg_original_ages))
        time.sleep(2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [89]:
stock_original_ages = []
stock_original_genders = []
stock_y_flipped_ages = []
stock_y_flipped_genders = []
for stock_image in os.listdir("stock_images"):
    if stock_image != ".DS_Store" and "transposed" not in stock_image:
        original, flipped  = detect(f"stock_images/{stock_image}")
        stock_original_ages.append(original['faces'][0]['attributes']['age']['value'])
        stock_original_genders.append(original['faces'][0]['attributes']['gender']['value'])
        stock_y_flipped_ages.append(flipped['faces'][0]['attributes']['age']['value'])
        stock_y_flipped_genders.append(flipped['faces'][0]['attributes']['gender']['value'])
        print(len(stock_original_ages))
        time.sleep(2)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100


In [90]:
images = stylegan_images
images = stylegan_images + stock_images

original_ages = sg_original_ages + stock_original_ages

original_genders = sg_original_genders + stock_original_genders

y_flipped_ages = sg_y_flipped_ages + stock_y_flipped_ages

y_flipped_genders  = sg_y_flipped_genders + stock_y_flipped_genders
len(original_ages)

200

In [93]:
images = stylegan_images + stock_images

In [103]:
stock_images_copy = []
for stock_image in os.listdir("stock_images"):
    if stock_image != ".DS_Store" and "transposed" not in stock_image:
        stock_images_copy.append(stock_image)
len(stock_images_copy)

100

In [104]:
images = stylegan_images + stock_images_copy
len(images)

200

In [106]:
len(original_ages)

200

In [107]:
df = pd.DataFrame()
df["Images"] = images
df["Original Age"] = original_ages
df["Original Gender"] = original_genders
df["Y-Axis Flip Age"] = y_flipped_ages
df["Y-Axis Flip Gender"] = y_flipped_genders
df

,Images,Original Age,Original Gender,Y-Axis Flip Age,Y-Axis Flip Gender
0,1628188590507_2_2_3.jpg,66,Male,72,Male
1,1628188590507_2_0_1.jpg,21,Female,22,Female
2,1628188590507_2_0_0.jpg,20,Female,20,Female
3,1628188590507_2_2_2.jpg,31,Male,31,Male
4,1628188590507_2_2_0.jpg,19,Female,17,Male
...,...,...,...,...,...
195,a_0_2_1.jpg,21,Male,21,Male
196,a_0_2_3.jpg,53,Male,51,Male
197,a_0_0_1.jpg,25,Female,29,Female
198,a_0_0_0.jpg,34,Female,36,Female


In [108]:
df.to_csv("y_axis_flip_data.csv")